In [1]:
import numpy as np

distance = np.load('distance_ILMI_1000+393_random_dtw_matrix.npy')
y_data = np.load('y_data_ILMI_1000+393_random_dtw_matrix.npy')
patient_ids = np.load('patient_ids_ILMI_1000+393_random_dtw_matrix.npy')

In [2]:
t = np.array(distance).reshape(distance.shape[0],12,12,1)
print(t.shape)

(6965, 12, 12, 1)


In [3]:
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import scale
from tensorflow.keras.layers import Flatten, Activation
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
import tensorflow.keras.backend as K
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPool2D, AvgPool2D
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score, f1_score
from sklearn.model_selection import GroupKFold
from sklearn.linear_model import SGDClassifier
import numpy as np


In [4]:
t.shape

(6965, 12, 12, 1)

In [8]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPool2D, Flatten, Dense
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.losses import BinaryCrossentropy
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score, f1_score
from sklearn.model_selection import GroupKFold
from sklearn.linear_model import SGDClassifier
from scipy.stats import mode
gkf = GroupKFold(n_splits=5)

sum_accuracy = 0
sum_sensitivity = 0
sum_specificity = 0
sum_f1 = 0

for train_idx, test_idx in gkf.split(t, y_data, groups=patient_ids):
    X_train, X_test = t[train_idx], t[test_idx]
    y_train, y_test = y_data[train_idx], y_data[test_idx]

    tf.keras.backend.clear_session()

    model = Sequential()

    K.clear_session()


    model = Sequential()

    model.add(Conv2D(10, (3, 3), input_shape=(12, 12, 1)))
    model.add(MaxPool2D(pool_size=(2, 2)))
    model.add(Flatten())

    # model.add(Dense(1024, activation='relu'))
    # model.add(Dense(512, activation='relu'))
    model.add(Dense(256, activation='relu'))
    model.add(Dense(10, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))

    model.compile(loss='BinaryCrossentropy',
                  optimizer='adam',
                  metrics=['accuracy'])

    model.summary()

    model.fit(X_train, y_train, batch_size=32,
              epochs=25, verbose=1, validation_split=0.25)
    
    predictions = model.predict(X_test)
    y_pred = np.round(predictions).astype(int).transpose()
    y_pred = y_pred[0]

    y_pred_majority = []
    for patient_idx in np.unique(patient_ids[test_idx]):
        segment_predictions = y_pred[patient_ids[test_idx] == patient_idx]
        majority_vote = mode(segment_predictions).mode[0]
        y_pred_majority.append(majority_vote)

    y_test_majority = []
    for patient_idx in np.unique(patient_ids[test_idx]):
        segment_predictions = y_test[patient_ids[test_idx] == patient_idx]
        majority_vote = mode(segment_predictions).mode[0]
        y_test_majority.append(majority_vote)

    accuracy = accuracy_score(y_test_majority, y_pred_majority)
    sum_accuracy += accuracy

    tn, fp, fn, tp = confusion_matrix(y_test_majority, y_pred_majority).ravel()

    sensitivity = tp / (tp + fn)
    specificity = tn / (tn + fp)
    f1 = f1_score(y_test_majority, y_pred_majority)

    sum_sensitivity += sensitivity
    sum_specificity += specificity
    sum_f1 += f1

    print(confusion_matrix(y_test_majority, y_pred_majority))
    print("Accuracy:", accuracy)
#     print("Sensitivity:", sensitivity)
#     print("Specificity:", specificity)
#     print("F1-Score:", f1)
#     print()

average_accuracy = sum_accuracy / 5
average_sensitivity = sum_sensitivity / 5
average_specificity = sum_specificity / 5
average_f1 = sum_f1 / 5

print("Average Accuracy:", average_accuracy)
print("Average Sensitivity:", average_sensitivity)
print("Average Specificity:", average_specificity)
print("Average F1-Score:", average_f1)


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 10, 10, 10)        100       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 5, 5, 10)         0         
 )                                                               
                                                                 
 flatten (Flatten)           (None, 250)               0         
                                                                 
 dense (Dense)               (None, 256)               64256     
                                                                 
 dense_1 (Dense)             (None, 10)                2570      
                                                                 
 dense_2 (Dense)             (None, 1)                 11        
                                                        

C:\Users\Pushpam\AppData\Local\Temp\ipykernel_18340\2474038497.py:56: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  majority_vote = mode(segment_predictions).mode[0]
C:\Users\Pushpam\AppData\Local\Temp\ipykernel_18340\2474038497.py:62: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoi

131/131 [==============================] - 1s 2ms/step - loss: 0.3939 - accuracy: 0.8355 - val_loss: 0.3131 - val_accuracy: 0.8701
Epoch 2/25
131/131 [==============================] - 0s 2ms/step - loss: 0.2736 - accuracy: 0.9011 - val_loss: 0.2681 - val_accuracy: 0.8952
Epoch 3/25
131/131 [==============================] - 0s 2ms/step - loss: 0.2524 - accuracy: 0.9102 - val_loss: 0.3088 - val_accuracy: 0.8837
Epoch 4/25
131/131 [==============================] - 0s 2ms/step - loss: 0.2288 - accuracy: 0.9126 - val_loss: 0.2699 - val_accuracy: 0.8981
Epoch 5/25
131/131 [==============================] - 0s 2ms/step - loss: 0.2030 - accuracy: 0.9270 - val_loss: 0.3056 - val_accuracy: 0.9009
Epoch 6/25
131/131 [==============================] - 0s 2ms/step - loss: 0.1827 - accuracy: 0.9354 - val_loss: 0.3190 - val_accuracy: 0.8722
Epoch 7/25
131/131 [==============================] - 0s 2ms/step - loss: 0.1625 - accuracy: 0.9397 - val_loss: 0.2887 - val_accuracy: 0.9060
Epoch 8/25
131/13

C:\Users\Pushpam\AppData\Local\Temp\ipykernel_18340\2474038497.py:56: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  majority_vote = mode(segment_predictions).mode[0]
C:\Users\Pushpam\AppData\Local\Temp\ipykernel_18340\2474038497.py:62: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoi

131/131 [==============================] - 1s 3ms/step - loss: 0.3701 - accuracy: 0.8382 - val_loss: 0.3832 - val_accuracy: 0.8378
Epoch 2/25
131/131 [==============================] - 0s 2ms/step - loss: 0.3038 - accuracy: 0.8836 - val_loss: 0.3226 - val_accuracy: 0.8708
Epoch 3/25
131/131 [==============================] - 0s 2ms/step - loss: 0.2953 - accuracy: 0.8834 - val_loss: 0.3218 - val_accuracy: 0.8859
Epoch 4/25
131/131 [==============================] - 0s 2ms/step - loss: 0.2542 - accuracy: 0.9028 - val_loss: 0.3230 - val_accuracy: 0.8830
Epoch 5/25
131/131 [==============================] - 0s 2ms/step - loss: 0.2440 - accuracy: 0.9050 - val_loss: 0.3543 - val_accuracy: 0.8658
Epoch 6/25
131/131 [==============================] - 0s 2ms/step - loss: 0.2275 - accuracy: 0.9141 - val_loss: 0.3282 - val_accuracy: 0.8744
Epoch 7/25
131/131 [==============================] - 0s 2ms/step - loss: 0.2215 - accuracy: 0.9107 - val_loss: 0.2987 - val_accuracy: 0.9002
Epoch 8/25
131/13

C:\Users\Pushpam\AppData\Local\Temp\ipykernel_18340\2474038497.py:56: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  majority_vote = mode(segment_predictions).mode[0]
C:\Users\Pushpam\AppData\Local\Temp\ipykernel_18340\2474038497.py:62: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoi

131/131 [==============================] - 1s 3ms/step - loss: 0.4808 - accuracy: 0.7869 - val_loss: 0.3791 - val_accuracy: 0.8364
Epoch 2/25
131/131 [==============================] - 0s 2ms/step - loss: 0.3394 - accuracy: 0.8708 - val_loss: 0.3180 - val_accuracy: 0.8802
Epoch 3/25
131/131 [==============================] - 0s 2ms/step - loss: 0.3051 - accuracy: 0.8828 - val_loss: 0.2992 - val_accuracy: 0.8881
Epoch 4/25
131/131 [==============================] - 0s 2ms/step - loss: 0.2700 - accuracy: 0.9055 - val_loss: 0.2921 - val_accuracy: 0.8910
Epoch 5/25
131/131 [==============================] - 0s 2ms/step - loss: 0.2477 - accuracy: 0.9084 - val_loss: 0.3170 - val_accuracy: 0.8637
Epoch 6/25
131/131 [==============================] - 0s 2ms/step - loss: 0.2275 - accuracy: 0.9134 - val_loss: 0.2706 - val_accuracy: 0.8981
Epoch 7/25
131/131 [==============================] - 0s 2ms/step - loss: 0.2067 - accuracy: 0.9227 - val_loss: 0.2718 - val_accuracy: 0.8967
Epoch 8/25
131/13

C:\Users\Pushpam\AppData\Local\Temp\ipykernel_18340\2474038497.py:56: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  majority_vote = mode(segment_predictions).mode[0]
C:\Users\Pushpam\AppData\Local\Temp\ipykernel_18340\2474038497.py:62: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoi

131/131 [==============================] - 1s 2ms/step - loss: 0.3535 - accuracy: 0.8433 - val_loss: 0.3215 - val_accuracy: 0.8637
Epoch 2/25
131/131 [==============================] - 0s 2ms/step - loss: 0.2782 - accuracy: 0.8897 - val_loss: 0.2787 - val_accuracy: 0.8960
Epoch 3/25
131/131 [==============================] - 0s 2ms/step - loss: 0.2444 - accuracy: 0.9053 - val_loss: 0.2809 - val_accuracy: 0.9017
Epoch 4/25
131/131 [==============================] - 0s 2ms/step - loss: 0.2121 - accuracy: 0.9177 - val_loss: 0.2839 - val_accuracy: 0.8938
Epoch 5/25
131/131 [==============================] - 0s 2ms/step - loss: 0.2091 - accuracy: 0.9196 - val_loss: 0.3211 - val_accuracy: 0.8938
Epoch 6/25
131/131 [==============================] - 0s 2ms/step - loss: 0.1769 - accuracy: 0.9352 - val_loss: 0.3083 - val_accuracy: 0.8845
Epoch 7/25
131/131 [==============================] - 0s 2ms/step - loss: 0.1636 - accuracy: 0.9381 - val_loss: 0.2629 - val_accuracy: 0.9003
Epoch 8/25
131/13

C:\Users\Pushpam\AppData\Local\Temp\ipykernel_18340\2474038497.py:56: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  majority_vote = mode(segment_predictions).mode[0]
C:\Users\Pushpam\AppData\Local\Temp\ipykernel_18340\2474038497.py:62: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoi

In [40]:
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GroupKFold
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import GroupKFold
import tensorflow as tf
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPool2D, Flatten, Dense
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.losses import BinaryCrossentropy

# Assuming t is a list of 4 matrices, each of shape (12, 12)
# Combine the 4 matrices into a single 4-channel input
X_data = np.stack(t, axis=-1)

gkf = GroupKFold(n_splits=5)

sum_accuracy = 0
sum_sensitivity = 0
sum_specificity = 0

for train_idx, test_idx in gkf.split(t, y_data, groups=patient_ids):
    X_train, X_test = t[train_idx], t[test_idx]
    y_train, y_test = y_data[train_idx], y_data[test_idx]

    tf.keras.backend.clear_session()

    model = Sequential()

    K.clear_session()


    model = Sequential()

    model.add(Conv2D(10, (3, 3), input_shape=(12, 12, 1)))
    model.add(MaxPool2D(pool_size=(2, 2)))
    model.add(Flatten())

    # model.add(Dense(1024, activation='relu'))
    # model.add(Dense(512, activation='relu'))
#     model.add(Dense(256, activation='relu'))
    model.add(Dense(128, activation='relu'))
    model.add(Dense(10, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))

    model.compile(loss='BinaryCrossentropy',
                  optimizer='adam',
                  metrics=['accuracy'])

    model.fit(X_train, y_train, batch_size=32,
              epochs=25, verbose=1, validation_split=0.25)

    predictions = model.predict(X_test)
    y_pred = np.round(predictions).astype(int).transpose()
    y_pred = y_pred[0]

    y_pred_majority = []
    for patient_idx in np.unique(patient_ids[test_idx]):
        segment_predictions = y_pred[patient_ids[test_idx] == patient_idx]
        majority_vote = np.bincount(segment_predictions).argmax()
        y_pred_majority.append(majority_vote)

    y_test_majority = []
    for patient_idx in np.unique(patient_ids[test_idx]):
        segment_predictions = y_test[patient_ids[test_idx] == patient_idx]
        majority_vote = np.bincount(segment_predictions).argmax()
        y_test_majority.append(majority_vote)

    accuracy = accuracy_score(y_test_majority, y_pred_majority)
    sum_accuracy += accuracy
  
    tn, fp, fn, tp = confusion_matrix(y_test_majority, y_pred_majority).ravel()

    sensitivity = tp / (tp + fn)
    specificity = tn / (tn + fp)

    sum_sensitivity += sensitivity
    sum_specificity += specificity

    print(confusion_matrix(y_test_majority, y_pred_majority))
    print("Accuracy:", accuracy)
    print("Sensitivity:", sensitivity)
    print("Specificity:", specificity)

average_accuracy = sum_accuracy / 5
average_sensitivity = sum_sensitivity / 5
average_specificity = sum_specificity / 5

print("Average Accuracy:", average_accuracy)
print("Average Sensitivity:", average_sensitivity)
print("Average Specificity:", average_specificity)


Epoch 1/25
131/131 [==============================] - 1s 2ms/step - loss: 0.4568 - accuracy: 0.8051 - val_loss: 0.3908 - val_accuracy: 0.8177
Epoch 2/25
131/131 [==============================] - 0s 2ms/step - loss: 0.3297 - accuracy: 0.8628 - val_loss: 0.3513 - val_accuracy: 0.8392
Epoch 3/25
131/131 [==============================] - 0s 2ms/step - loss: 0.2835 - accuracy: 0.8963 - val_loss: 0.2962 - val_accuracy: 0.8909
Epoch 4/25
131/131 [==============================] - 0s 2ms/step - loss: 0.3150 - accuracy: 0.8717 - val_loss: 0.3021 - val_accuracy: 0.8737
Epoch 5/25
131/131 [==============================] - 0s 2ms/step - loss: 0.2560 - accuracy: 0.9040 - val_loss: 0.2885 - val_accuracy: 0.8758
Epoch 6/25
131/131 [==============================] - 0s 2ms/step - loss: 0.2423 - accuracy: 0.9100 - val_loss: 0.2679 - val_accuracy: 0.8988
Epoch 7/25
131/131 [==============================] - 0s 2ms/step - loss: 0.2285 - accuracy: 0.9150 - val_loss: 0.2756 - val_accuracy: 0.8952
Epoch 

131/131 [==============================] - 0s 2ms/step - loss: 0.2110 - accuracy: 0.9164 - val_loss: 0.2933 - val_accuracy: 0.8959
Epoch 7/25
131/131 [==============================] - 0s 2ms/step - loss: 0.2022 - accuracy: 0.9241 - val_loss: 0.2679 - val_accuracy: 0.9131
Epoch 8/25
131/131 [==============================] - 0s 2ms/step - loss: 0.1766 - accuracy: 0.9313 - val_loss: 0.2566 - val_accuracy: 0.9275
Epoch 9/25
131/131 [==============================] - 0s 2ms/step - loss: 0.1680 - accuracy: 0.9385 - val_loss: 0.2800 - val_accuracy: 0.9031
Epoch 10/25
131/131 [==============================] - 0s 2ms/step - loss: 0.1474 - accuracy: 0.9449 - val_loss: 0.2999 - val_accuracy: 0.9146
Epoch 11/25
131/131 [==============================] - 0s 2ms/step - loss: 0.1492 - accuracy: 0.9452 - val_loss: 0.2970 - val_accuracy: 0.9081
Epoch 12/25
131/131 [==============================] - 0s 2ms/step - loss: 0.1318 - accuracy: 0.9512 - val_loss: 0.2944 - val_accuracy: 0.9031
Epoch 13/25
13

In [ ]:
def extract_upper_triangular(matrix):
    n = len(matrix)
    upper_triangular_array = []

    for i in range(n):
        for j in range(i, n):
            upper_triangular_array.append(matrix[i][j])

    return upper_triangular_array

def concatenate_upper_triangular_vectors(matrix1, matrix2, matrix3, matrix4):
    vector1 = extract_upper_triangular(matrix1)
    vector2 = extract_upper_triangular(matrix2)
    vector3 = extract_upper_triangular(matrix3)
    vector4 = extract_upper_triangular(matrix4)

    concatenated_vector = vector1 + vector2 + vector3 + vector4
    return concatenated_vector

t2=[]
for i in range(distance.shape[0]):
    K_i = extract_upper_triangular(distance[i])
    t2.append(K_i)
t2 = np.array(t2)
print(t2.shape)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(t, y_data,
                                                    test_size=0.25,
                                                    random_state=42)

K.clear_session()


model = Sequential()

model.add(Conv2D(10, (3, 3), input_shape=(12, 12, 1)))
model.add(MaxPool2D(pool_size=(2, 2)))
model.add(Flatten())

# model.add(Dense(1024, activation='relu'))
# model.add(Dense(512, activation='relu'))
model.add(Dense(256, activation='relu'))
model.add(Dense(10, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='BinaryCrossentropy',
              optimizer='adam',
              metrics=['accuracy'])

model.summary()

model.fit(X_train, y_train, batch_size=32,
          epochs=25, verbose=1, validation_split=0.25)

model.evaluate(X_test, y_test)

from sklearn.metrics import confusion_matrix

y_pred = model.predict(X_test)
y_pred_binary = (y_pred > 0.5).astype(int) 

confusion = confusion_matrix(y_test, y_pred_binary)

TN, FP, FN, TP = confusion.ravel()

specificity = TN / (TN + FP)
sensitivity = TP / (TP + FN)

accuracy = (TP + TN) / (TP + TN + FP + FN)

print(f'Specificity: {specificity}')
print(f'Sensitivity: {sensitivity}')
print(f'Accuracy: {accuracy}')

